In [2]:
from pathlib import Path
import json

TEXT = Path("/Users/joregan/Playing/wolne-match/text")
WHISPER = Path("/Users/joregan/Playing/wolne-match/whisperx")

In [ ]:
!pip install mosestokenizer

In [7]:
test = TEXT / "zloczynca.txt"

textlines = []
with open(test, "r") as f:
    for line in f:
        line = line.strip()
        if line == "":
            continue
        textlines.append(line)

In [9]:
test2 = "/Users/joregan/Playing/wolne-match/whisperx/zloczynca_000.json"

def read_whisperx_json(filename):
    with open(filename) as f:
        data = json.load(f)
    return data

In [18]:
from mosestokenizer import MosesSentenceSplitter

def get_sentences_poorly(text):
    outputs = []
    with MosesSentenceSplitter("pl") as splitter:
        for line in text:
            sentences = splitter([line])
            outputs.extend(sentences)
    return outputs

In [28]:
def rejoin_poorly_split_sentences(sentences):
    outputs = []
    i = 0
    while i < len(sentences) - 1:
        if sentences[i].endswith(":") and sentences[i + 1][0].islower():
            outputs.append(sentences[i] + " " + sentences[i + 1])
            i += 2
        else:
            outputs.append(sentences[i])
            i += 1
    return outputs

In [ ]:
import re

def continue_splitting(sentence):
    pattern = r'(?<=[.?!]) — (?=[A-ZĄĆĘŁŃÓŚŹŻ])'
    parts = re.split(pattern, sentence)
    return parts

In [61]:
sentence = "'— Nigdy nie kłamałem, a teraz od razu niby kłamię… — mruczy Denis, mrugając oczyma. — Czyż można bez ciężarka, panie sędzio?'"
continue_splitting(sentence)

["'— Nigdy nie kłamałem, a teraz od razu niby kłamię… — mruczy Denis, mrugając oczyma.",
 "Czyż można bez ciężarka, panie sędzio?'"]

In [ ]:
rejoin_poorly_split_sentences(get_sentences_poorly(textlines))

In [ ]:
def clean_word(word):
    word = word.strip(",;:!?—…„”\"“.«»*()[]‘/\\")
    if word:
        return word.lower()
    else:
        return ""

def clean_text(text):
    words = text.split(" ")
    cleaned_words = []
    for word in words:
        word = word.strip(",;:!?—…„”\"“.«»*()[]‘/\\")
        if word:
            cleaned_words.append(word.lower())
    return " ".join(cleaned_words)

In [66]:
clean_text("Jego zarośnięta, dziobata twarz i oczy, ledwo widoczne spoza gęstych, obwisłych brwi, mają wyraz posępnej surowości.")

'jego zarośnięta dziobata twarz i oczy ledwo widoczne spoza gęstych obwisłych brwi mają wyraz posępnej surowości'

In [ ]:
for segment in read_whisperx_json(test2)["segments"]:
    cleaned = clean_text(segment["text"].strip(" "))
    print(cleaned)